In [ ]:
from src.all_in_one import *

In [ ]:
cds = pd.read_csv('consolidated_dataset3.csv')

In [ ]:
# Intake dataset
# psint_file = path_manager.get_data_file(Category.PS, PSFile.PSNT)
# psint_dataset = Dataset(config_file, psint_file.sheet)
# psint = get_df(psint_file.path, psint_file.sheet, config_file).sort_values(by='nt_s1')

In [ ]:
# psint_0 = psint.drop(columns=['nt_re', 'note']).dropna(subset=['nt_s1'])  # Intake assessment without re intake
# psint_0 = psint_0.drop_duplicates(subset=['rid'], keep='first')  # This will drop a beneficiary with R0062 - This rid has two beneficiaries! 

In [ ]:
# psreint = psint[['rid', 'nt_re']].dropna()

In [ ]:
# Consolidate Intake and re-intake
# cpsint = pd.merge(psint_0, psreint, on='rid', how='left')
# reint = cpsint['nt_re']
# cpsint.drop(columns=['nt_re'], inplace=True)
# cpsint.insert(9, 'nt_re', reint)
# cpsint

In [ ]:
# cds_int = pd.merge(cds, cpsint, on='rid', how='left') 

In [ ]:
# cds_int.to_csv('consolidated_dataset1.csv', index=False)

In [ ]:
# Group Counseling dataset
# psg_file = path_manager.get_data_file(Category.PS, PSFile.PSG)
# psg_dataset = Dataset(config_file, psg_file.sheet)
# psg = get_df(psg_file.path, psg_file.sheet, config_file).drop_duplicates(subset=['rid'])  # This drop duplicates will drop a beneficiary that has rid of another one.
# psg = psg.drop(columns=['note'])


In [ ]:
# cds_g = pd.merge(cds, psg, on='rid', how='left')
# cds_g.to_csv('consolidated_dataset2.csv', index=False)

In [ ]:
# Individual Counseling dataset
# psi_file = path_manager.get_data_file(Category.PS, PSFile.PSI)
# psi_dataset = Dataset(config_file, psi_file.sheet)
# psi = get_df(psi_file.path, psi_file.sheet, config_file)
# psi = get_df(psi_file.path, psi_file.sheet, config_file).drop_duplicates(subset=['rid'])  # This drop duplicates will drop a beneficiary that has rid of another one.
# psi = psi.drop(columns=['note'])

In [ ]:
# psi = psi.rename(columns={'fcid': 'icfcid', 'firstname': 'icfirstname', 'lastname': 'iclastname', 'need_pt': 'icneed_pt'})
# cds_i = pd.merge(cds, psi, on='rid', how='left')
# cds_i.to_csv('consolidated_dataset3.csv', index=False)

In [ ]:
# PS Follow-up dataset
# psfu_file = path_manager.get_data_file(Category.PS, PSFile.PSFU)
# psfu_dataset = Dataset(config_file, psfu_file.sheet)
# psfu = get_df(psfu_file.path, psfu_file.sheet, config_file).drop_duplicates(subset=['rid'])  # There are two duplicates found.
# psfu = psfu.drop(columns=['note'])
# psfu = psfu.rename(
#     columns={'fcid': 'psfufcid', 'firstname': 'psfufirstname', 'lastname': 'psfulastname'})

In [ ]:
# cds_fu = pd.merge(cds, psfu, on='rid', how='left')

In [ ]:
# cds_fu.to_csv('consolidated_dataset4.csv', index=False)